In [2]:
import pandas as pd
import numpy as np
import time
import json

import tqdm #lib para progress bar
import glob #outras libs pra mexer com os: os, fnmatch, pathlib

import requests as rq
import bs4

In [3]:
df = pd.read_json('parsed_videos.json', lines=True)
#lines=True diz que cada linha do json tem um linha de dados
df.sort_values('link').tail(30)
#Curso de Inteligência Artificial é uma sujeira que veio na extração dos dados

,title,link,query,page
170,Machine Learning Tutorial | Learn Machine Lear...,/watch?v=zNhTOPQeRaU,machine+learning,9
839,A data science como a nova ciência social defi...,/watch?v=zNoLeZi5gpk,data+science,12
1043,What is Data Science ll Jobs ll Career opportu...,/watch?v=zOY7RtmMRJQ,data+science,23
369,MIT 6.S091: Introduction to Deep Reinforcement...,/watch?v=zR11FLZ-O9M,machine+learning,19
933,MLOps: How to Bring Your Data Science to Produ...,/watch?v=zgXvZzt9C-c,data+science,16
1156,Become a Citizen Data Scientist | Allison Sagr...,/watch?v=zgijAGkdjZc,data+science,28
1137,What a Typical Data Science Project Looks Like...,/watch?v=zkOiwTu9Qpw,data+science,27
501,Geoffrey Hinton: The Foundations of Deep Learning,/watch?v=zl99IZvW7rE,machine+learning,25
628,Curso de Inteligência Artificial,https://www.googleadservices.com/pagead/aclk?s...,data+science,2
343,Curso de Inteligência Artificial,https://www.googleadservices.com/pagead/aclk?s...,machine+learning,18


In [4]:
links_list = df['link'].unique()
len(links_list)

1174

In [36]:
url = 'https://www.youtube.com{}'

for link in tqdm.notebook.tqdm(links_list):
    if '?v=' in link:
    #para excluir os
        url_f = url.format(link)
        response = rq.get(url_f)
        video_id = link[9:]
        with open('./raw_data/video_{}.html'.format(video_id), 'w+', encoding="utf-8") as video:
            video.write(response.text)
        time.sleep(2)

In [12]:
videos_path_list = glob.glob('./raw_data/video*') 
#videos_path_list = glob.iglob('./raw_data/video*') 
#retorna a list de glob.glob() como um iterator, mas a progress bar não funciona com um iterator

In [23]:
with open("parsed_video_info.json", 'w+') as output:
    for video_file in tqdm.tqdm_notebook(videos_path_list):
        try:
            with open(video_file, 'r+', encoding="utf-8") as inp:
                html_page = inp.read()
                parsed = bs4.BeautifulSoup(html_page, 'html.parser')
            
                data = {}
            
                #title
                title = parsed.find_all('span', attrs={'id':'eow-title'})
                data['title'] = title[0].text.strip()
                #.strip() para remover os espaços do começo e fim do title
            
                #link
                
                data['link'] = 'https://www.youtube.com/watch?v='+video_file[17:]
                #[:17] para remover o './raw_data\\video_'
            
                #number of subscribers
                num_subs = parsed.find_all(attrs={'class':'yt-subscription-button-subscriber-count-branded-horizontal yt-subscriber-count'})
                data['subscribers'] = num_subs[0]['aria-label']
                #num_subs[0] porque .find_all retorna uma list
            
                #views
                views = parsed.find_all(attrs={'class':'watch-view-count'})[0]
                views_num = views.text.split()[0]
                data['views'] = views_num
                #.split()[0] porque views com como '### visualizações'
            
                #number of links
                likes_button = parsed.find_all(attrs={'class':'like-button-renderer-like-button-unclicked'})
                tag_like = likes_button[0].find_all('span')
                data['likes'] = tag_like[0].text.replace('.','')
                #likes_button[0] e tag_like[0] porque .find_all retorna uma list
            
                #number of dislike
                dislikes_button = parsed.find_all(attrs={'class':'like-button-renderer-dislike-button-clicked'})
                tag_dislike = dislikes_button[0].find_all('span')
                data['dislikes'] = tag_dislike[0].text.replace('.','')
                #likes_button[0] e tag_dislike[0] porque .find_all retorna uma list
            
                #date
                date = parsed.find_all(attrs={'class':'watch-time-text'})[0]
                data['date'] = date.text
                
                #duration
                duration_code = parsed.find_all(attrs={'itemprop':"duration"})[0]
                data['duration_code'] = duration_code['content']
                
                #thumbnailUrl
                thumb = parsed.find_all(attrs={'itemprop':'thumbnailUrl'})[0]
                data['thumbnailUrl'] = thumb['href']
                
                #outras ideias para melhoria:
                #channel
                #category
                #video_definition
                #comments_num
            
                output.write('{}\n'.format(json.dumps(data)))
        except:
            pass

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [24]:
len(videos_path_list) - pd.read_json('parsed_video_info.json', lines=True).shape[0]
#arquivos que não foram processados, futuramente investigar a causa de tantos erros

95

In [25]:
data_without_label = pd.read_json('parsed_video_info.json', lines=True)
data_without_label

,title,link,subscribers,views,likes,dislikes,date,duration_code,thumbnailUrl
0,Machine Learning Course A To Z || Beginner to ...,https://www.youtube.com/watch?v=-58kO_zYUGE.html,173 mil,175.311,5131,65,Publicado em 10 de ago. de 2018,PT495M58S,https://i.ytimg.com/vi/-58kO_zYUGE/maxresdefau...
1,How to Become A Machine Learning Engineer | Ho...,https://www.youtube.com/watch?v=-5hEYRt8JE0.html,484 mil,34.724,782,20,Publicado em 3 de set. de 2018,PT9M44S,https://i.ytimg.com/vi/-5hEYRt8JE0/maxresdefau...
2,Python For Data Science Full Course - 9 Hours ...,https://www.youtube.com/watch?v=-6RqxhNO2yY.html,"1,62 mi",18.021,680,8,Publicado em 15 de mar. de 2020,PT562M56S,https://i.ytimg.com/vi/-6RqxhNO2yY/maxresdefau...
3,Michael I. Jordan: Machine Learning: Dynamical...,https://www.youtube.com/watch?v=-8yYFdV5SOc.html,"2,84 mil",4.193,94,3,Publicado em 2 de mai. de 2019,PT75M13S,https://i.ytimg.com/vi/-8yYFdV5SOc/maxresdefau...
4,Best Programming Languages for Machine Learning,https://www.youtube.com/watch?v=-cdxxrbKdho.html,702 mil,103.756,3167,155,Publicado em 5 de ago. de 2018,PT8M21S,https://i.ytimg.com/vi/-cdxxrbKdho/hqdefault.jpg
...,...,...,...,...,...,...,...,...,...
1052,R Programming Tutorial - Learn the Basics of S...,https://www.youtube.com/watch?v=_V8eKsto3Ug.html,"1,87 mi",632.150,10542,465,Publicado em 6 de jun. de 2019,PT130M39S,https://i.ytimg.com/vi/_V8eKsto3Ug/maxresdefau...
1053,Day in the Life: Data Scientist,https://www.youtube.com/watch?v=_Wk9T_G-u4o.html,"27,9 mil",93.006,1181,36,Publicado em 12 de fev. de 2018,PT2M36S,https://i.ytimg.com/vi/_Wk9T_G-u4o/hqdefault.jpg
1054,Online Courses for Data Science Specialization,https://www.youtube.com/watch?v=_XbttSk3ALs.html,148 mil,79.924,3133,45,Publicado em 28 de ago. de 2019,PT16M53S,https://i.ytimg.com/vi/_XbttSk3ALs/maxresdefau...
1055,[Lập trình Machine learning cơ bản] - Bài 1: G...,https://www.youtube.com/watch?v=_ZjIv2D6T40.html,"78,1 mil",8.947,195,5,Publicado em 21 de jan. de 2020,PT20M18S,https://i.ytimg.com/vi/_ZjIv2D6T40/maxresdefau...


In [28]:
data_without_label.to_csv('data_without_label.csv')